In [ ]:
pip install skyfield

In [1]:
import pandas as pd
import skyfield
import re

from IPython.display import clear_output
from skyfield.api import EarthSatellite, utc, wgs84, load
from datetime import datetime, timedelta

In [2]:
logDir = 'L:/DATA/ISIS/ISIS_Test_Metadata_Analysis/MERGED/ISIS-2_MD_Version6.csv'

In [3]:
data = pd.read_csv('L:/DATA/ISIS/Orbitcheck/isis_2_tle.csv')
data.head()

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:38:47,18 SPCS,ISIS 2,1971-024A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1971-04-01,AFWTR,NaN,33974,9022795,0 ISIS 2,1 05104U 71024 A 71091.33272721 .00002038 +0...,2 05104 088.1564 293.4291 0045120 139.7159 220...
1,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:38:47,18 SPCS,ISIS 2,1971-024A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1971-04-01,AFWTR,NaN,33974,9022796,0 ISIS 2,1 05104U 71024 A 71095.12186674 .00002038 +0...,2 05104 088.1528 292.7933 0045674 131.8713 228...
2,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:38:47,18 SPCS,ISIS 2,1971-024A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1971-04-01,AFWTR,NaN,33974,9022797,0 ISIS 2,1 05104U 71024 A 71097.33220238 .00002038 +0...,2 05104 088.1522 292.4379 0044859 124.7781 235...
3,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:38:47,18 SPCS,ISIS 2,1971-024A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1971-04-01,AFWTR,NaN,33974,9022798,0 ISIS 2,1 05104U 71024 A 71099.93723390 .00002038 +0...,2 05104 088.1511 292.0156 0045675 120.0596 240...
4,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:38:47,18 SPCS,ISIS 2,1971-024A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1971-04-01,AFWTR,NaN,33974,9022799,0 ISIS 2,1 05104U 71024 A 71101.91074869 .00002038 +0...,2 05104 088.1577 291.6901 0046374 113.5666 247...


In [4]:
tle = data.loc[:, ["EPOCH", "TLE_LINE1", "TLE_LINE2"]]
tle.EPOCH = pd.to_datetime(tle.EPOCH)

In [5]:
ISIS2 = pd.read_csv(logDir)
ISIS2.head()

,Outlier,Fixed_Frequency_Code,Year,Day,Hour,Minute,Second,Filename,Directory,Subdirectory,...,Adjusted_Satellite_Code,Adjusted_Station_Number,Station_quality,Station_Location_y,Station_ID_y,Number,Station_Latitude,Station_Longitude,Latitude,Longitude
0,NaN,5,1971,228,7,22,44,Image0001.png,b11_R014207871,B1-35-20 ISIS B D-409,...,4,69,1,"Tromso, Norway",TRO,69,69.39,18.56,69.7N,18.9E
1,NaN,5,1971,228,7,29,43,Image0002.png,b11_R014207871,B1-35-20 ISIS B D-409,...,4,69,1,"Tromso, Norway",TRO,69,69.39,18.56,69.7N,18.9E
2,NaN,5,1971,226,7,29,55,Image0003.png,b11_R014207871,B1-35-20 ISIS B D-409,...,4,69,1,"Tromso, Norway",TRO,69,69.39,18.56,69.7N,18.9E
3,NaN,5,1971,228,1,30,9,Image0004.png,b11_R014207871,B1-35-20 ISIS B D-409,...,4,69,1,"Tromso, Norway",TRO,69,69.39,18.56,69.7N,18.9E
4,NaN,5,1971,226,1,30,23,Image0005.png,b11_R014207871,B1-35-20 ISIS B D-409,...,4,69,1,"Tromso, Norway",TRO,69,69.39,18.56,69.7N,18.9E


In [6]:
# function for settling quadrant ambiguity (before realized skyfield has NWES values available)

def quad_latlon(coords):
    splt_str = re.split(r'(\d+\.\d+)', coords)
    new_coord = float(''.join(splt_str[1:-1]))
    
    quad = splt_str[-1:]
    
    if quad == ["N"]:
        return new_coord
    elif quad == ["S"]:
        return new_coord*-1
    elif quad == ["E"]:
        return new_coord
    elif quad == ["W"]:
        return new_coord*-1
    else:
        print(splt_str)
        print("An Error has Occured")
        return 0

In [8]:
# function for getting the nearest TLE record

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [9]:
result_dir = []
result_sub_dir = []
result_name = []

result_type = []

result_read_epoch = []
result_read_station = [] 
result_tle_epoch = []

result_flag = []

result_events = []
result_event_times = []

result_alt = []
result_distance =[] 

result_lat = []
result_lon = []
result_height = []

for i in range(len(ISIS1)):
    record_epoch = ISIS2.loc[i, "Timestamp"]
    try: 
        record_epoch = datetime.strptime(record_epoch, '%Y.%m.%d %H:%M:%S')
    except: 
        record_epoch = datetime.datetime(1971, 0, 0, 0, 0, 0)
    station = ISIS2.iloc[i, -2:]
    quality = ISIS2.loc[i, "Station_quality"]

    nearest_epoch = nearest(tle.EPOCH, record_epoch)
    nearest_tle = tle.loc[tle['EPOCH'] == nearest_epoch]
    
    station = wgs84.latlon(quad_latlon(station.iloc[0]), quad_latlon(station.iloc[1]))
    
    ts = load.timescale()
    satellite = EarthSatellite(nearest_tle.iat[0,1], nearest_tle.iat[0,2], 'ISIS-2', ts)
    
    d = record_epoch.replace(tzinfo=utc)
    t = ts.utc(d)
    
    geocentric = satellite.at(t)
    lat, lon = wgs84.latlon_of(geocentric)
    height = wgs84.height_of(geocentric)
    
#    difference between station
    difference = satellite - station

    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()
    
    if alt.degrees > 0:
        flag = 0 
    else:
        flag = 1
    
    if quality == 1:
        t0 = t - 0.010416666666666666
        t1 = t + 0.010416666666666666
    elif quality == 2: 
        continue 
        t0 = t - 0.020833333333333332
        t1 = t + 0.020833333333333332
    elif quality == 3:
        t0 = t
        t1 = t + 0.041667
    elif quality == 4: 
        t0 = t
        t1 = t + 1
        
    event_time, events = satellite.find_events(station, t0, t1, altitude_degrees=15.0)
    event_time = event_time.utc_strftime('%Y %b %d %H:%M:%S')

    result_dir.append(ISIS2.loc[i, "Directory"])
    result_sub_dir.append(ISIS2.loc[i, "Subdirectory"])
    result_name.append(ISIS2.loc[i, "Filename"])

    
    result_read_epoch.append(record_epoch)
    result_read_station.append(ISIS2.loc[i, "Station_Location_y"])
    result_tle_epoch.append(nearest_epoch)

    result_flag.append(flag)

    result_events.append(events)
    result_event_times.append(event_time)

    result_alt.append(alt.degrees)
    result_distance.append(distance.km)

    result_lat.append(lat.degrees)
    result_lon.append(lon.degrees)
    result_height.append(height.km)

    clear_output(wait=True)
    print(i)
    
print("Procsesing Complete")

dict = {'Directory': result_dir, 'Subdirectory': result_sub_dir, 'Filename': result_name,
        'Read_Epoch': result_read_epoch, 'Read_Station': result_read_station, 'TLE_Epoch': result_tle_epoch,
        'Flag': result_flag, 'Events': result_events, 'Event_Times': result_event_times,
        'Station_Alt': result_alt, 'Station_Distance': result_distance,
        'Sat_Lat': result_lat, 'Sat_Lon': result_lon, 'Sat_Height': result_height} 
        
        
df = pd.DataFrame(dict)
     
# saving the dataframe
df.to_csv('L:/DATA/ISIS/Orbitcheck/ISIS-2_orbitcheck.csv')

99
Procsesing Complete
